In [3]:
!pip install sentence-transformers

     |████████████████████████████████| 78 kB 1.7 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 3.4 MB 58.5 MB/s            
     |████████████████████████████████| 6.8 MB 16.7 MB/s            
     |████████████████████████████████| 881.9 MB 7.4 kB/s             
     |████████████████████████████████| 23.2 MB 131 kB/s            
     |████████████████████████████████| 1.5 MB 81.8 MB/s            
     |████████████████████████████████| 1.2 MB 36.3 MB/s            
     |████████████████████████████████| 67 kB 1.4 MB/s             
     |████████████████████████████████| 3.3 MB 77.8 MB/s            
     |████████████████████████████████| 895 kB 30.5 MB/s            
     |████████████████████████████████| 763 kB 47.1 MB/s            
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=5d97805085d08f83f04180a9ead1737ce3783d613813bc68457a2e9db335c6f9
  Stored in directo

In [4]:
import pandas as pd

from sentence_transformers import SentenceTransformer, models

2022-01-19 09:07:53.060623: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-19 09:07:53.060657: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
###### CREATE MODEL ######
max_seq_length = 128
train_batch_size = 128

# Load teacher model
print("Load teacher model")
teacher_model = SentenceTransformer('stsb-roberta-base-v2')

# Create student model
print("Create student model")
word_embedding_model = models.Transformer("xlm-roberta-base")

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Load teacher model


Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/675 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [6]:
train_batch_size = 128

In [7]:
#### Read Datasets ####

df1 = pd.read_csv('EN_TL_compiled_400k.csv')
df2 = pd.read_csv('STS_TL_400k.csv')
#df3 = pd.read_csv('datasets/translate_s2.txt', delimiter = '\t', encoding = 'unicode_escape')


In [8]:
df1

,tl,en
0,"Ayon naman kay Bayan Muna Rep. Carlos Zarate, ...","According to Bayan Muna Rep. Carlos Zarate, Ch..."
1,Ayon naman sa Philippine Embassy sa Washington...,According to the Philippine Embassy in Washing...
2,Paano maipagtatanggol ng mga ordinaryong mamam...,How do ordinary citizens defend themselves fro...
3,SM by the bay,SM by the bay
4,"MANILA, Philippines - Inirekomenda ng Departme...","Manila, Philippines - The Department of Interi..."
...,...,...
399995,"""The President will discuss with Japanese Prim...","""The President Will Discuss With Japanese Prim..."
399996,Matatandaang idiniskwalipika ng Commission on ...,The Commission on Elections (COMELEC) (COMELEC...
399997,"Dahil sa huling score, inilarawan ng Heritage ...","Due to the last score, the Heritage Foundation..."
399998,"MANILA, Philippines -- Wala umanong karapatan ...","Manila, Philippines - The Government of Puerto..."


In [9]:
df2

,s1,s2,label
0,"""Hindi ko ugali ang mamulitika; mas gusto kong...","Ito ang dineklara ni Atty. Romulo Macalintal, ...",1
1,"Ayon naman kay Bayan Muna Rep. Carlos Zarate, ...",Dating itinutulak ni Duterte ang pagbabago ng ...,0
2,Ayon naman sa Philippine Embassy sa Washington...,"Ayon sa NBI, hindi umano siyento por siyentong...",1
3,Paano maipagtatanggol ng mga ordinaryong mamam...,"""Nakasaad sa R.A. 9009 na dapat mayroong land ...",1
4,SM by the bay,Kabilang sa mga sumali sa programang ito ang m...,1
...,...,...,...
419995,2p.m. Arellano University vs Hog's Breath Cafe,Inirekomenda sa Pangulo ni Albay Rep. Joey Sal...,1
419996,Jason Davee: We rather want the senate head to...,"Sabi naman ni Eberl, gumagana pa rin naman uma...",1
419997,Ipinatupad ang liquid ban sa mga istasyon ng M...,Gumastos naman ng P11.587 milyon ang mga contr...,1
419998,Team Standings: zArellano (13-4); zSan Beda (1...,z - Final Four twice-to-beat,0


In [10]:
#df3

In [11]:
from sentence_transformers.datasets import ParallelSentencesDataset
from torch.utils.data import DataLoader
from sentence_transformers import SentencesDataset, losses, evaluation, readers


###### Load train sets ######

train_reader = ParallelSentencesDataset(student_model=model, teacher_model=teacher_model)
train_reader.add_dataset(df1.values.tolist())
#load_data('translate_s1.txt', encoding= 'unicode_escape')
train_dataloader = DataLoader(train_reader, shuffle=True, batch_size=train_batch_size)
train_loss = losses.MSELoss(model=model)

In [12]:
###### Load test sets Mean Squared Error (MSE) measures ######

evaluators = []

test_mse = evaluation.MSEEvaluator(df1['en'].values.tolist(),df1['tl'].values.tolist(), teacher_model=teacher_model, batch_size=train_batch_size, show_progress_bar=True, write_csv = True)
evaluators.append(test_mse)

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

In [13]:
###### Load dev sets for Semantic Textual Similarity (STS) data ######

evaluator_sts = evaluation.EmbeddingSimilarityEvaluator(df2['s1'], df2['s2'], df2['label'], batch_size=train_batch_size, show_progress_bar=True, write_csv=True)
evaluators.append(evaluator_sts)

In [14]:
import tensorflow.compat.v1 as tf
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')

2022-01-19 09:18:03.546259: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-19 09:18:03.546423: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-01-19 09:18:03.546491: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-01-19 09:18:03.546557: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-01-19 09:18:03.546620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

In [15]:
strategy = tf.distribute.MirroredStrategy(gpus)
tf.config.get_visible_devices()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [17]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0","/gpu:1","/gpu:2", "/gpu:3"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [21]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [22]:
###### Train model ######

import datetime

#from numba import jit, cuda, numba

#@cuda.jit

#with tf.device("/gpu:3"):
with mirrored_strategy.scope():
    output_path = "output/model-" + datetime.date.today().strftime("%Y-%m-%d")
    model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[-1]),
          epochs=100,
          evaluation_steps=1000,
          warmup_steps=10000,
          scheduler='warmupconstant',
          output_path=output_path,
          save_best_model=True,
          optimizer_params= {'lr': 2e-5, 'eps': 1e-6, 'correct_bias': False}
          )



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1522 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.76 GiB total capacity; 9.24 GiB already allocated; 16.44 MiB free; 9.43 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [20]:
evaluators

In [ ]:
import scipy.spatial

#Corpus with example sentences
corpusEN = df3['EN'][:20].values.tolist()
corpusTL = df3['TL'][:20].values.tolist()

with strategy.scope():
    corpusEN_embeddings = model.encode(corpusEN)
    corpusTL_embeddings = model.encode(corpusTL)

In [ ]:
queries = df1['EN'][88:89].values.tolist()
query_embeddings = model.encode(queries)

In [ ]:
closest_n = 5

with strategy.scope():
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpusEN_embeddings, "cosine")[0]
    
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
    
        print("\n=======\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:\n")

        for idx, distance in results[0:closest_n]:
            print(corpusEN[idx].strip(), "(Score: %.4f)" % (1-distance))

In [ ]:
queriesTL = df1['TL'][88:89].values.tolist()
#queriesTL = ['gobyerno.']
queryTL_embeddings = model.encode(queriesTL)

In [ ]:
closest_n = 5
for query, queryTL_embedding in zip(queriesTL, queryTL_embeddings):
    distances = scipy.spatial.distance.cdist([queryTL_embedding], corpusTL_embeddings, "cosine")[0]
    
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    
    print("\n=======\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:\n")

    for idx, distance in results[0:closest_n]:
        print(corpusTL[idx].strip(), "(Score: %.4f)" % (1-distance))

### after 100 epocs of training: Query Search Result

In [ ]:
import scipy.spatial

#Corpus with example sentences
corpusEN = df3['EN'][:20].values.tolist()
corpusTL = df3['TL'][:20].values.tolist()

with strategy.scope():
    corpusEN_embeddings = model.encode(corpusEN)
    corpusTL_embeddings = model.encode(corpusTL)

In [ ]:
queries = df1['EN'][88:89].values.tolist()
query_embeddings = model.encode(queries)

In [ ]:
closest_n = 5

with strategy.scope():
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpusEN_embeddings, "cosine")[0]
    
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
    
        print("\n=======\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:\n")

        for idx, distance in results[0:closest_n]:
            print(corpusEN[idx].strip(), "(Score: %.4f)" % (1-distance))

In [ ]:
queriesTL = df1['TL'][88:89].values.tolist()
#queriesTL = ['Balita tungkol sa transportasyon, gobyerno ng Pilipinas at iba pa.']
queryTL_embeddings = model.encode(queriesTL)

In [ ]:
closest_n = 5
for query, queryTL_embedding in zip(queriesTL, queryTL_embeddings):
    distances = scipy.spatial.distance.cdist([queryTL_embedding], corpusTL_embeddings, "cosine")[0]
    
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    
    print("\n=======\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:\n")

    for idx, distance in results[0:closest_n]:
        print(corpusTL[idx].strip(), "(Score: %.4f)" % (1-distance))